In [165]:
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
from plotly.subplots import make_subplots
import datetime as dt


import numpy as np
import pandas as pd
import os
import math
import json

In [381]:
path = "../data/ontario/cases.jsonl"
with open(path, 'r') as outfile:
    all_cases = [ json.loads(c) for c in outfile]
df =  pd.DataFrame.from_dict(all_cases)

In [382]:
cities = list(df.city.unique())

infections = [[case["date"][:10] for case in all_cases if case["city"]== city and case["date"][:10] > "2020-02-01"] for city in cities ]


In [383]:
path = "../data/ontario/all_updates.json"
with open(path, 'r') as outfile:
    updates = [ json.loads(c) for c in outfile]
    


In [384]:
updates = sorted(updates, key=lambda k: k['date']) 

dates = { t['date'][:10]:i for i, t in enumerate(updates)}

unique_dates_ix = list(dates.values())
dates = list(dates.keys())

In [385]:


negative = [t['negative'] for i,t in enumerate(updates) if i in unique_dates_ix]
under = [t['pending'] for i,t in enumerate(updates)  if i in unique_dates_ix]
positive = [t['positive'] for i,t in enumerate(updates)  if i in unique_dates_ix]
resolved = [t['resolved'] for i,t in enumerate(updates) if i in unique_dates_ix]
dead = [t['deceased'] for i,t in enumerate(updates)  if i in unique_dates_ix]

total_df = pd.DataFrame.from_dict(updates)

In [386]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=positive,
                    mode='lines',
                    name='Total Number of Cases'))

fig.update_layout(
    title_text='Total Number of Cases', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
)
fig.show()

with open('../plotly/ontario_total_cases.json', 'w') as f:
    f.write(fig.to_json())

In [404]:
doubling = [ (i+1)*math.log(2)/(math.log(p/positive[0])) for i,p in enumerate(positive[1:])]
fig = go.Figure(data=go.Scatter(x=dates[1:], y=doubling,
                    mode='lines',
                    name='Time to Double'))
fig.update_layout(title='Current Doubling Time',
                   xaxis_title='Date',
                   yaxis_title='Days to Double')
fig.show()
with open('../plotly/ontario_doubling.json', 'w') as f:
    f.write(fig.to_json())

In [411]:

fig = go.Figure(data=[
    go.Bar(name='Negative', x=dates, y=negative),
    go.Bar(name='Under Investigation', x=dates, y=under),
    go.Bar(name='Confirmed Positive', x=dates, y=positive),
    go.Bar(name='Resolved', x=dates, y=resolved),
    go.Bar(name='Dead', x=dates, y=dead),
])
fig.update_layout(
    title_text='Infection over time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.7, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

with open('../plotly/ontario_infection.json', 'w') as f:
    f.write(fig.to_json())

TypeError: to_json() got an unexpected keyword argument 'responsive'

In [409]:
import collections
city_infections = []
cities.remove("pending")
cities.remove("Unspecified")
infections = [[case["date"][:10] for case in all_cases if case["city"]== city and case["date"][:10] > "2020-02-01"] for city in cities]


for infection in infections : 
    sorted_inf = sorted(infection)
    counter = collections.Counter(sorted_inf)
    for d in dates : 
        if d not in counter: 
            counter[d] = 0
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[0])}
    values = list(counter.values())
    city_infections.append( list(np.cumsum(values)))

In [414]:
color = ['#EB89B5', '#5b7bd6','#ff6960', '#d11411', '#00b159', '#00aedb', '#f37735', '#ffc425',
          '#a200ff', '#f47835', '#d41243', '#8ec127', '#feda75', '#fa7e1e', '#962fbf']*2


#fig = go.Figure()
fig = make_subplots(rows=8, cols=5, subplot_titles=tuple(cities), shared_xaxes=True, shared_yaxes = True)

for  i,m, in enumerate(city_infections) :
    r = i//5+1
    c = i%5+1
    fig.add_trace(
    go.Scatter(x=dates, y=m),
    row=r, col=c
)
    #fig.add_trace(go.Histogram( x=m, name = c, marker_color = co))

#fig.update_layout(barmode='stack')

#fig = px.histogram(df, x="date", color = "city", color_discrete_sequence= px.colors.qualitative.Light24)
# fig.update_layout(
#     title_text='New Infections over Time', # title of plot
#     xaxis_title_text='Date', # xaxis label
#     yaxis_title_text='Infected', # yaxis label
#     bargap=0.2, # gap between bars of adjacent location coordinates
#     bargroupgap=0.1 # gap between bars of the same location coordinates
# )
fig.update_layout(showlegend=False, title_text="Infection over time per City", height=1000)
fig.show()

with open('../plotly/ontario_new_infections_by_city.json', 'w') as f:
    f.write(fig.to_json())

In [390]:

import math
df = pd.DataFrame.from_dict(all_cases)

male = [int(math.ceil(case["age"] / 10.0)) * 10 for case in all_cases if case["gender"] == "Male" and case["age"] != 'pending']


#male = [ {"age" :p["age"], "city" : p["city"]} for p in all_cases if p["gender"] == "Male" and p["age"] != 'pending' ]
#male = pd.DataFrame.from_dict(male)
          
#female = [ {"age" :p["age"], "city" : p["city"]} for p in all_cases if p["gender"] == "Female" and p["age"] != 'pending']
#female = pd.DataFrame.from_dict(female)

female = [case["age"] for case in all_cases if case["gender"] == "Female" and case["age"] != 'pending'] 


In [391]:

color = ['#EB89B5', '#5b7bd6','#ff6960', '#d11411', '#00b159', '#00aedb', '#f37735', '#ffc425',
          '#a200ff', '#f47835', '#d41243', '#8ec127', '#feda75', '#fa7e1e', '#962fbf']*2
fig = go.Figure()
fig.add_trace(go.Histogram(x=male, marker_color = color[0]))
# for  m, c, co in zip(male, cities, color) : 
#     fig.add_trace(go.Histogram(x=m, name = c, marker_color = co))

#fig.update_layout(barmode='stack')

fig.update_layout(
    title_text='Male Infection by Age', # title of plot
    xaxis_title_text='Age ', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

with open('../plotly/ontario_male_infection.json', 'w') as f:
    f.write(fig.to_json())

In [392]:
color = ['#EB89B5', '#5b7bd6','#ff6960', '#d11411', '#00b159', '#00aedb', '#f37735', '#ffc425',
          '#a200ff', '#f47835', '#d41243', '#8ec127', '#feda75', '#fa7e1e', '#962fbf']*2
fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Histogram(x=female, marker_color = color[0]))


# for  m, c, co in zip(female, cities, color) : 
#     fig.add_trace(go.Histogram(x=m, name = c, marker_color = co))

# fig.update_layout(barmode='stack')

fig.update_layout(
    title_text='Female Infection by Age', # title of plot
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    #bargap=0.2, # gap between bars of adjacent location coordinates
    #bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

with open('../plotly/ontario_female_infection.json', 'w') as f:
    f.write(fig.to_json())

In [393]:


transmission = ['travel', 'close contact', 'pending']
infections = []
for t in transmission:
    temp = []
    for case in all_cases: 
        if "transmission" in case:
            if t in case['transmission']:
                temp.append(case["date"][:10])
    infections.append(temp)
            



In [396]:
fig = go.Figure()
for  m, t in zip(infections, transmission) : 
    fig.add_trace(go.Histogram( x=m, name = t))

fig.update_layout(barmode='stack')

#fig = px.histogram(df, x="date", color = "city", color_discrete_sequence= px.colors.qualitative.Light24)
fig.update_layout(
    title_text='New transmission over time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [394]:
import collections
cum_infections = []

for infection in infections[:2] : 
    sorted_inf = sorted(infection)
    counter = collections.Counter(sorted_inf)
    for d in dates : 
        if d not in counter: 
            counter[d] = 0
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[0])}
    values = list(counter.values())
    cum_infections.append( list(np.cumsum(values)))

In [395]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=cum_infections[0],
                    mode='lines',
                    name='Travel'))
fig.add_trace(go.Scatter(x=dates, y=cum_infections[1],
                    mode='lines',
                    name='Close Contact'))
fig.update_layout(
    title_text='Cumulative Transmission over Time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Number Infected', # yaxis label
)

fig.show()

with open('../plotly/ontario_transmission_type.json', 'w') as f:
    f.write(fig.to_json())